In [7]:
import tensorflow as tf
import pandas as pd
import xlsxwriter 
import codecs
import pickle
import datetime
import import_ipynb
from Parameter_Define import *
from CV_Preprocessing import *
from CV_JD_Matching import *

In [6]:
def _getToday():
        return datetime.date.today().strftime("%Y%m%d")

In [3]:
pkl_filename = "Model_Saved/MSBI_Resume_Model.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [4]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [13]:
PREDICT_FOLDER =parameters_values('predict')
Threshhold=0.80
Job_Desc=parameters_values('jd')
pre_df,pre_file_names = read_resumes('predicted', PREDICT_FOLDER)
Jd_df,Jd_file_names = read_resumes('JD', Job_Desc)
values=JD_Matching(pre_df,Jd_df)
pre_file_names=[line.split("/")[-1] for line in pre_file_names]
Jd_file_names=[line.split("/")[-1] for line in Jd_file_names]
values.columns = Jd_file_names
values["Candidate Names"] = pre_file_names
c="%s_%s.%s" % ("Model_Saved/Shortlisted_Candidates_", _getToday() ,"csv")
values.to_csv(c)
#pre_df["File_Name"]=pre_file_names
#data = pd.read_csv("Model_Saved/Shortlisted_Candidates.csv")
#mergedDf = pre_df.merge(data, on='File_Name')
#mergedDf.drop( mergedDf[ mergedDf['Flag'] == "Fail" ].index , inplace=True)


In [ ]:
cols = values.columns.tolist()
cols = cols[-1:] + cols[:-1]
values= values[cols] 

In [10]:
values

,JD for Lead.docx,JD for Analyst.docx,Candidate Names
0,0.44,0.43,Ravinder Kumar.doc
1,0.56,0.50,D Suresh Babu .doc
2,0.34,0.33,Archana Bhat.doc
3,0.49,0.44,Manoj Kumar Yadav.doc
4,0.54,0.54,Pawan Hedau_Sql Developer.doc
5,0.53,0.52,R Venkatesulu.doc
6,0.54,0.53,SaurabhKumbhar_SQL Developer.docx
7,0.53,0.50,Jagadeesh.docx
8,0.42,0.37,Shib Shankar Santra_sql Developer.docx
9,0.51,0.48,Ritesh Kumar.docx


In [31]:
#Preprocess X
max_words = 1200
t = tf.keras.preprocessing.text.Tokenizer(num_words = max_words)
t.fit_on_texts(pre_df["Text"])
x = t.texts_to_matrix(pre_df["Text"], mode = 'count')

In [49]:
for i in pre_df:
    #predict_y=pickle_model.predict_classes(x)
    pred_y= pickle_model.predict(x)
    pr= pickle_model.predict_proba(x)
    predict_file_names=[line.split("/")[-1] for line in pre_file_names]


In [59]:
selection_prob=[]
rejection_prob=[]
for q,w in pr:
      rejection_prob.append(str(truncate(q*100,2))+'%')
      selection_prob.append(str(truncate(w*100,2))+'%')

In [61]:
unsorted_prediction = [(j, predict_file_names[j],str(selection_prob[j]),"Selected" if pred_y[j] == 1 else "Rejected") for j in range(len(pr))]

In [62]:
Prediction = sorted(unsorted_prediction, key=lambda x: 1 if x[2] == 'Rejected' else -1)

In [64]:
pf = pd.DataFrame(Prediction,columns=['S.No.','Candidate Names','Predicitive Score(in %)','Status'])
today = date.today()
d1 = today.strftime("%Y/%m/%d")
pf.to_csv('Predicted_Data/Predicted_Candidates_'+ d1+'.csv',index=False)